# Block Cipher
## AES con CBC Y ECB
### Cifrado de Información
#### José Daniel Gómez Cabrera 21429

In [59]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import os
import base64
from PIL import Image
import io

In [60]:
def aes_generate_key():
    # AES-256 utiliza una clave de 32 bytes (256 bits)
    return get_random_bytes(32)

In [61]:
def aes_generate_iv():
    # AES usa bloques de 16 bytes, por lo que el IV también es de 16 bytes
    return get_random_bytes(AES.block_size)

In [62]:
def aes_encrypt_cbc(plaintext, key, iv=None):
    # Usar IV proporcionado o generar uno nuevo
    if iv is None:
        iv = aes_generate_iv()
    # Crear objeto cifrador AES en modo CBC
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    # Aplicar padding
    padded_data = pad(plaintext.encode('utf-8') if isinstance(plaintext, str) else plaintext, AES.block_size)
    # Cifrar los datos
    ciphertext = cipher.encrypt(padded_data)
    # Retornar IV + ciphertext en base64
    return base64.b64encode(iv + ciphertext), iv

In [63]:
def aes_decrypt_cbc(ciphertext, key):
    # Decodificar de base64
    ciphertext = base64.b64decode(ciphertext)
    # Extraer IV (primeros 16 bytes)
    iv = ciphertext[:16]
    ciphertext = ciphertext[16:]
    # Crear objeto descifrador AES en modo CBC
    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    # Descifrar
    padded_plaintext = cipher.decrypt(ciphertext)
    # Eliminar padding
    plaintext = unpad(padded_plaintext, AES.block_size)
    # Retornar como texto
    return plaintext.decode('utf-8')

In [64]:
def aes_encrypt_ecb(plaintext, key):
    # Crear objeto cifrador AES en modo ECB
    cipher = AES.new(key, AES.MODE_ECB)
    # Aplicar padding
    padded_data = pad(plaintext.encode('utf-8') if isinstance(plaintext, str) else plaintext, AES.block_size)
    # Cifrar los datos
    ciphertext = cipher.encrypt(padded_data)
    # Retornar en base64
    return base64.b64encode(ciphertext)

In [65]:
def aes_decrypt_ecb(ciphertext, key):
    # Decodificar de base64
    ciphertext = base64.b64decode(ciphertext)
    # Crear objeto descifrador AES en modo ECB
    cipher = AES.new(key, AES.MODE_ECB)
    # Descifrar
    padded_plaintext = cipher.decrypt(ciphertext)
    # Eliminar padding
    plaintext = unpad(padded_plaintext, AES.block_size)
    # Retornar como texto
    return plaintext.decode('utf-8')

## Cifrado de imagenes

In [66]:
def encrypt_image_aes(image_path, key, mode="CBC"):
    """Cifra una imagen usando AES en modo CBC o ECB"""
    # Leer imagen
    with open(image_path, 'rb') as f:
        image_data = f.read()

    if mode == "CBC":
        # Cifrar con CBC
        iv = aes_generate_iv()
        cipher = AES.new(key, AES.MODE_CBC, iv=iv)
        padded_data = pad(image_data, AES.block_size)
        encrypted_data = iv + cipher.encrypt(padded_data)
    else:
        # Cifrar con ECB
        cipher = AES.new(key, AES.MODE_ECB)
        padded_data = pad(image_data, AES.block_size)
        encrypted_data = cipher.encrypt(padded_data)

    # Guardar imagen cifrada
    encrypted_path = image_path.split('.')[0] + f'_encrypted_{mode}.' + image_path.split('.')[-1]
    with open(encrypted_path, 'wb') as f:
        f.write(encrypted_data)

    return encrypted_path

In [67]:
def decrypt_image_aes(encrypted_path, key, mode="CBC"):
    """Descifra una imagen usando AES en modo CBC o ECB"""
    # Leer imagen cifrada
    with open(encrypted_path, 'rb') as f:
        encrypted_data = f.read()

    if mode == "CBC":
        # Extraer IV (primeros 16 bytes)
        iv = encrypted_data[:16]
        encrypted_data = encrypted_data[16:]
        # Descifrar con CBC
        cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    else:
        # Descifrar con ECB
        cipher = AES.new(key, AES.MODE_ECB)

    # Descifrar y eliminar padding
    padded_data = cipher.decrypt(encrypted_data)
    image_data = unpad(padded_data, AES.block_size)

    # Guardar imagen descifrada
    decrypted_path = encrypted_path.split('_encrypted')[0] + f'_decrypted_{mode}.' + encrypted_path.split('.')[-1]
    with open(decrypted_path, 'wb') as f:
        f.write(image_data)

    return decrypted_path

In [68]:
# Leer el archivo
with open("./data/des.txt", "r") as f:
    plaintext = f.read()

print("Texto original:", plaintext)
print("-" * 50)

Texto original: The DES block cipher is a 16-round Feistel network with a block length of
64 bits and a key length of 56 bits. The same round function ˆ f is used in each
of the 16 rounds. The round function takes a 48-bit sub-key and, as expected
for a (balanced) Feistel network, a 32-bit input (namely, half a block). The
key schedule of DES is used to derive a sequence of 48-bit sub-keys k1, . . . , k16
from the 56-bit master key.
--------------------------------------------------


In [69]:
print("AES con CBC:")
aes_key = aes_generate_key()
aes_encrypted_cbc, iv = aes_encrypt_cbc(plaintext, aes_key)
print("Texto cifrado con CBC (base64):", aes_encrypted_cbc.decode())
aes_decrypted_cbc = aes_decrypt_cbc(aes_encrypted_cbc, aes_key)
print("Texto descifrado:", aes_decrypted_cbc)
print("¿Coincide con el original?", end="")
print("Si" if aes_decrypted_cbc == plaintext else "No")

AES con CBC:
Texto cifrado con CBC (base64): GuukgHHbEtnr6b/ia4pqiA/vdtjfk6+RT3WmV3FxGzHBnX+lhH4yI2GHoMxj9Cgm8aqCcazp+Lx0g9/hu20J7MkGtyDYjvlOR08fb83nG3SClST2o+ht/Oo/CZh6+WHkNGsjvOSrUxYQh8dx39/7skOoAU5yGiMCw4iD3AJTU0TtB/rT6dSUUxFEjdTMpJvmNjKkJOOmCadT2UwNqkzQJRHIGpEW582OhdyWcRulItDQlMULsbC+mDJQOvqR1GsEm6KTVG31xJ2CrX18A8onigc9/RhUxXNQno+VoFedl2aoGD/9h9WTOcKdRdOx/u7Qg+Rh5/XhB9AZHaJTXV8k5jABshCzBZVnnPkAW+ujL72tb8s7I/RRplNi5dSDeYFfDXoGSEpfvszTtkiW/8qL7OhcieaUTQKUXJhMThWvMnxsqxyLI5olXCSOkQlpKVw1zfTcG9j4+KxC2Hs4/9RmXqHogImCBcsz+IH2QpjDgJL8Lh1nS4ba6mZ+cktGdEb+kxrXRYyriodP2fMHRWtGEO1VpdKuWjZUjbrAJZExkrCro8nKEj1HteyPLKlxrMvTX5dlSppqOianL13/kZud9A==
Texto descifrado: The DES block cipher is a 16-round Feistel network with a block length of
64 bits and a key length of 56 bits. The same round function ˆ f is used in each
of the 16 rounds. The round function takes a 48-bit sub-key and, as expected
for a (balanced) Feistel network, a 32-bit input (namely, half a block). The
key schedule of DES is used

In [70]:
print("AES con ECB:")
aes_encrypted_ecb = aes_encrypt_ecb(plaintext, aes_key)
print("Texto cifrado con ECB (base64):", aes_encrypted_ecb.decode())
aes_decrypted_ecb = aes_decrypt_ecb(aes_encrypted_ecb, aes_key)
print("Texto descifrado:", aes_decrypted_ecb)
print("¿Coincide con el original?", "Si" if aes_decrypted_ecb == plaintext else "No")

AES con ECB:
Texto cifrado con ECB (base64): Qf7snP0J6LaA8DvW6A0Ai9TGanjywovIE7lEj4UMP/dG0ChpE1PQ6u3ptWIlBW/7C8q/TovcFBIDHpgGAC01zb+llKU5Ql2c2NSO9bwMfe7iWVTarXE5bHUisPMbWMQxVmgosXwLcBkRr7ABZJiVFKqXS5CmLgKlj2kD2MYpim1KxpTLS94OT4ycM3QKe7Al61SqiAmtw/UGtjCRRvmITw0HbiifnO1VROU9hBJB/2nVe4D+Itm5Uu/0z9OY6KmjwSiF8F0A71op9tEypZvxNb7DrSE4snYR43Ybw0iGDY4Y1gal9xnoOQk6SdzAyWatNY2/TM4gHdAmhJ3v8hl2/6DT1TYOygn/TTjAtA00/Q5MeEypO15ZH6yeb0urDYR0FLPMdgl8M3ryFhBkJA0vn/U2Rur9KDTCEmnqyLI2h4ckGEeP3q0UANeQxSNFXdm3IdRc8R3F0AFOuH/SlZ9vHApoIDBWiydnzizL8EWFwWT+Pleiu17fytD91s71necwUgrB/yl5XwKfujvwno9rnHGli60KaDDFbgwEFTU8T78aFARcU+ZFhVb8rDHRVR4/
Texto descifrado: The DES block cipher is a 16-round Feistel network with a block length of
64 bits and a key length of 56 bits. The same round function ˆ f is used in each
of the 16 rounds. The round function takes a 48-bit sub-key and, as expected
for a (balanced) Feistel network, a 32-bit input (namely, half a block). The
key schedule of DES is used to derive a sequence of